# 02 - Variography
Calcula variograma experimental y genera modelo simple.

In [1]:
import os, sys, json

PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
if os.path.basename(os.getcwd()) == "notebooks":
    os.chdir(PROJECT_ROOT)
else:
    PROJECT_ROOT = os.getcwd()

sys.path.insert(0, PROJECT_ROOT)

In [2]:
from src.io_utils import read_csv_robust, standardize_columns, coerce_numeric
from src.variography import experimental_variogram_2d, fit_variogram_model, plot_variogram, save_variogram_model

with open("config/project.json", "r", encoding="utf-8") as f:
    cfg = json.load(f)

if not os.path.exists(cfg["data_csv_path"]):
    from src.make_demo_data import make_demo_csv
    make_demo_csv(cfg["data_csv_path"])

df = read_csv_robust(cfg["data_csv_path"])

mapping = {
    "x": cfg["columns"].get("x"),
    "y": cfg["columns"].get("y"),
    "z": cfg["columns"].get("z"),
    "var": cfg["columns"].get("variable_objetivo"),
}

df = standardize_columns(df, mapping)
coerce_numeric(df, ["x", "y", "z", "var"])
df.dropna(subset=["x", "y", "var"], inplace=True)

exp = experimental_variogram_2d(df, "x", "y", "var", cfg["variogram"])
model = fit_variogram_model(exp, float(df["var"].var(ddof=1)), model_type="spherical")

os.makedirs("outputs/figures", exist_ok=True)
os.makedirs("outputs/models", exist_ok=True)
plot_variogram(exp, model, "outputs/figures/variogram.png")
save_variogram_model(model, "outputs/models/variogram_var.json")

 x,y,z offsets = 0.0,50.0
